# Entrenar Modelo de Accesibilidad (CodeT5) en Google Colab

Este cuaderno te permite entrenar tu modelo de IA sin usar los recursos de tu PC local. 
Utilizará las GPUs gratuitas de Google (Tesla T4 o similar).

### Pasos:
1. Sube tu archivo `mi_dataset_wcag_gen.csv` a los archivos de este entorno (Icono de carpeta a la izquierda -> Subir).
2. Ejecuta todas las celdas en orden (Menú 'Entorno de ejecución' -> 'Ejecutar todas').
3. Al finalizar, se descargará automáticamente un archivo `modelo_entrenado.zip`.
4. Descomprime ese archivo en tu carpeta `python-agent/` reemplazando la carpeta existente `mi_modelo_wcag_generativo`.

In [ ]:
# 1. Instalar dependencias necesarias
!pip install transformers datasets accelerate torch pandas scikit-learn

In [ ]:
# 2. Script de Entrenamiento (Copiado de tu proyecto local)
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
import os

# Desactivar WandB (Weights & Biases) para que no pida login
os.environ["WANDB_DISABLED"] = "true"

# Verificar si el CSV existe
if not os.path.exists('mi_dataset_wcag_gen.csv'):
    raise FileNotFoundError("¡ERROR! No has subido el archivo 'mi_dataset_wcag_gen.csv'. Súbelo usando el panel de la izquierda.")

print("Cargando dataset...")
df = pd.read_csv("mi_dataset_wcag_gen.csv")
dataset = Dataset.from_pandas(df)

# Dividir train/test
dataset = dataset.train_test_split(test_size=0.1)

print("Descargando modelo base (Salesforce/codet5-small)...")
checkpoint = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

def preprocess_function(examples):
    inputs = ["fix wcag: " + code for code in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)
    labels = tokenizer(examples["target_text"], max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Procesando datos...")
tokenized_datasets = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./resultados_colab",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Colab T4 aguanta batch 8 o 16
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=15,            # Mantenemos 15 épocas
    predict_with_generate=True,
    fp16=torch.cuda.is_available(), # Usar GPU acelerada si está disponible
    logging_steps=50,
    report_to='none', # Aseguramos que no intente reportar a WandB
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("¡Iniciando entrenamiento en la nube!...")
trainer.train()

print("Guardando modelo...")
model.save_pretrained("./mi_modelo_wcag_generativo")
tokenizer.save_pretrained("./mi_modelo_wcag_generativo")
print("¡Modelo guardado correctamente!")

In [ ]:
# 3. Comprimir y Descargar
!zip -r modelo_entrenado.zip mi_modelo_wcag_generativo

from google.colab import files
files.download('modelo_entrenado.zip')